# NLP - TP Final

Augusto Rabbia

# Inicializar datos y Modelos

In [ ]:
import pandas as pd
import zipfile
import os

# LLM para generación
from huggingface_hub import InferenceClient

# Sentence Embedding
from transformers import AutoTokenizer, AutoModel
import torch

import requests

Obtenemos los datos del archivo .zip

In [ ]:
with zipfile.ZipFile('Datos TP1.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        if os.path.basename(root).startswith('.'): # Ignorar ipynb checkpoint
            continue
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
list_files('datos')

datos/
    queries_clasificador.csv
    informacion/
        review_bgg.txt
        video3.txt
        manual.txt
        video1.txt
        video2.txt
        review_externa.txt
        df_foros_bgg.csv
        video4.txt
    relaciones/
        creditos_relaciones.csv
    estadisticas/
        estadisticas.csv


El modelo de Phi-2 de Microsoft se utilizará para generación de texto a lo largo del trabajo.



In [ ]:
API_KEY = ""

class LanguageModel():
    def __init__(self, init_function, generation_function):
        self._model = init_function()
        self._generation_function = generation_function

    def generar_respuesta(self, prompt):
        return self._generation_function(self._model, prompt)

def init_mistral():
    client = InferenceClient(
        provider="novita",  # Puede cambiar si el proveedor es distinto
        api_key=API_KEY,
    )
    return client

def gen_mistral(client, prompt):
    response = client.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.3",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    return response.choices[0].message.content

llm_mistral = LanguageModel(init_mistral, gen_mistral)

Para embedding, se utilizarán AutoTokenizer y `multilingual-e5-small` a lo largo del trabajo.

In [ ]:
model_name = "intfloat/multilingual-e5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
embedder = AutoModel.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

# Crear Bases de Datos de Búsqueda

## BD Vectorial de Texto

Se hará embedding del texto utilizando el modelo `e5-multilingual-small` y la BD vectorial será utilizando ChromaDB.

Instalar e importar librerías

In [ ]:
!pip install chromadb --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.8 MB/s eta 

In [ ]:
import numpy as np
import pandas as pd

# Splitting de texto
from langchain.text_splitter import RecursiveCharacterTextSplitter

# BD Vectorial
import chromadb

Importar datos de texto

In [ ]:
df_foros_bgg = pd.read_csv('datos/informacion/df_foros_bgg.csv')
print(df_foros_bgg.shape)
df_foros_bgg.head()

(36, 3)


,Category,Link to Post,Conversation
0,Reviews,https://boardgamegeek.com/thread/2660465/revie...,Title: Review: Still fun after ten plays? My p...
1,Reviews,https://boardgamegeek.com/thread/2201438/tiny-...,Title: Tiny review of Tiny Towns\nOriginal pos...
2,Reviews,https://boardgamegeek.com/thread/2258475/teens...,Title: Teensy Towns (a Space-Biff! review)\nOr...
3,Reviews,https://boardgamegeek.com/thread/2208048/clear...,"Title: Clear Eyes, Tiny Towns, Can't Lose: A R..."
4,Reviews,https://boardgamegeek.com/thread/2291815/simpl...,"Title: Simple, Thinky and Brutal\nOriginal pos..."


In [ ]:
textos = df_foros_bgg['Conversation'].tolist()

for roota, dirs, files in os.walk('datos/informacion'):
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(roota, file), 'r') as f:
                textos.append(f.read())
len(textos)

43

Text splitting

In [ ]:
chunk_size = 1024
chunk_overlap = 128
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=[".", "?", "!","\n"]
    )

textos_split = []
for texto in textos:
    textos_split.append(text_splitter.split_text(texto))

textos_split = [item for sublist in textos_split for item in sublist]
len(textos_split), textos_split[0]

(204,
 "Title: Review: Still fun after ten plays? My pros and cons\nOriginal post: Even though long extensive reviews seem to be the most popular reviews on BGG, I myself just don't have the time or the patience to read those.I prefer quick overviews where I can get some insights into why this game may or may not be attractive to me.My reviews are targeted at a similar audience, and just consist of my pros and cons coming from:1) First impressions (before playing my first game)2) Impressions after the first game3) Current feelings (after at least 10 plays)First impressions:+ There was quite a lot of hype around this game when it was released, which drew my attention (the BGG crowd doesn’t often overhype a game that I end up hating)+ I like the box art and the components very much+ This is a bingo-style game, in which everybody places a cube of the same colour each turn, but make their own choices on the patterns in which they are placed, which are used to place buildings on the 4x4 gri

Embedding de oraciones

In [ ]:
class BDVectorial():
    def __init__(self, tokenizer, embedder, sentences):
        self._model_name = model_name
        self._tokenizer = tokenizer
        self._embedder = embedder

        self._sentences = sentences
        self._embedded_sentences = self._embed_sentences(self._sentences)

        self._chroma_client = chromadb.PersistentClient(path="./BD_vectorial")
        self._collection = self._chroma_client.get_or_create_collection(name="sentence_embeddings")
        self._collection.add(
            documents=self._sentences,
            embeddings=self._embedded_sentences.tolist(),
            ids=[f"id_{i}" for i in range(len(self._embedded_sentences))]
        )

    def _embed_sentences(self, sentences):
        inputs = self._tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            model_output = self._embedder(**inputs)
        embeddings = model_output.last_hidden_state.mean(dim=1)  # Mean pooling
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)  # L2 normalize
        return embeddings.cpu().numpy()

    def buscar(self, texto, n_resultados=5):
        texto_embed = self._embed_sentences([texto])
        results = self._collection.query(
            query_embeddings=texto_embed.tolist(),
            n_results=n_resultados
        )
        return results['documents'][0]

In [ ]:
bd_vectorial = BDVectorial(tokenizer, embedder, textos_split)
query_prueba = "Cómo ganar en este juego?"
bd_vectorial.buscar(query_prueba)

['quieres un juego donde todos los jugadores juegan a la vez con muchas decisiones y un montón de edificios monísimos hoy te traemos Tiny towns en 90 segundos en Tiny towns Tendremos que construir nuestras pequeñas ciudades colocando primero los recursos y después utilizándolos para levantar edificios en cada turno un jugador se convertirá en el maestro constructor podrá elegir Qué recursos recibirán todos los jugadores y cada uno decidiremos en cuál de las 16 casillas de nuestro tablero lo queremos colocar al hacerlo el objetivo será formar patrones que coincidan con los necesarios para construir un edificio concreto cuando un patrón esté completo podremos retirar los recursos que Lo componen Y colocar el edificio en una de las casillas donde estos se encontraban cada edificio nos dará unas habilidades o formas de puntuar diferentes por ejemplo algunos puntos harán dependiendo de las cosas que tengan cerca otros de acuerdo a lo que hay en su fila columna otros servirán para intercambi

## BD Tabular de Estadísticas

Con queries construidas por el modelo generativo de Gemini 2.5.

In [ ]:
df_estadisticas = pd.read_csv('datos/estadisticas/estadisticas.csv')
df_estadisticas

,Title,Value
0,Avg. Rating,7.211
1,No. of Ratings,"20,744"
2,Difficulty according to the community,2.06 / 5
3,Comments,"3,037"
4,Fans,"1,018"
5,Overall Rank,462
6,Family Rank,121
7,All Time Plays,"96,506"
8,This Month,96
9,Own,"34,184"


In [ ]:
class BDTabular():
    def __init__(self, data: pd.DataFrame, lm: LanguageModel):
        self._data = data
        self._lm = lm
        self._estructura = {
            "columnas": data.columns.tolist(),
            "tipos": data.dtypes.tolist(),
            "cantidad filas": data.shape[0],
        }
        self._prompt_base = self._crear_prompt_base()

    def _crear_prompt_base(self):
        return f"""
Eres un modelo encargado de convertir las consultas del usuario que vienen en lenguaje natural, convertirlas a código de pandas en python, la información del dataframe es la siguiente:
columnas: {self._estructura['columnas']}
tipos: {self._estructura['tipos']}
cantidad filas: {self._estructura['cantidad filas']}

La consulta del usuario es la siguiente:
{{}}

RESPONDE UNICAMENTE CON CÓDIGO PYTHON Y NADA MÁS, LA VARIABLE SE LLAMA "df". NO NOS DES EL CÓDIGO PARA ALMACENAR EL VALOR EN UNA VARIABLE, SINO DAME EL CÓDIGO LISTO PARA IMPRIMIR SU EJECUCIÓN
"""

    def ejecutar_consulta(self, consulta):
        df = self._data
        prompt = self._prompt_base.format(consulta)
        respuesta = self._lm.generar_respuesta(prompt).replace('```python', '').replace('```', '').strip()

        if respuesta == "None":
            return None
        try:
            return eval(respuesta)
        except Exception as e:
            print(f"Error al evaluar el código: {e}")
            return None

In [ ]:
bd_tabular = BDTabular(df_estadisticas, llm_mistral)

## BD de Grafos de Relaciones

Con rdflib

In [ ]:
!pip install rdflib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 27.6 MB/s eta 0:00:00


In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace

Leer y limpiar dataframe.

In [ ]:
df_relaciones = pd.read_csv('datos/relaciones/creditos_relaciones.csv')
df_relaciones['Objeto'] = df_relaciones['Objeto'].str.replace(r'[^a-zA-Z0-9]', '', regex=True)

In [ ]:
class BDGraph:
    def __init__(self, data: pd.DataFrame, lm: LanguageModel):
        self._data = data
        self._lm = lm
        self._graph = Graph()
        self._ns = Namespace("http://ejemplo.org/")
        self._estructura = {
            "columnas": data.columns.tolist(),
            "cantidad filas": data.shape[0],
        }
        self._cargar_grafo()
        self._prompt_base = self._crear_prompt_base()

    def _cargar_grafo(self):
        for _, row in self._data.iterrows():
            sujeto = URIRef(self._ns[str(row['Sujeto']).replace(" ", "_")])
            relacion = URIRef(self._ns[str(row['Relación']).replace(" ", "_")])
            objeto = URIRef(self._ns[str(row['Objeto']).replace(" ", "_")])
            self._graph.add((sujeto, relacion, objeto))

    def _crear_prompt_base(self):
        return """
Eres un modelo encargado de traducir preguntas del usuario escritas en lenguaje natural a consultas SPARQL.

Estructura del grafo:
- Cada relación es un triplete del tipo (Sujeto, Relación, Objeto).
- Los nodos provienen de un dataframe con las columnas: {self._estructura['columnas']}.

Ejemplo de una consulta válida:
SELECT ?sujeto WHERE {{
    ?sujeto <http://ejemplo.org/es_amigo_de> <http://ejemplo.org/Juan> .
}}

Ahora, responde SOLO con la consulta SPARQL correspondiente a la siguiente pregunta del usuario:
{}

NO EXPLIQUES NADA. SOLO DEVUELVE LA CONSULTA SPARQL.
"""

    def ejecutar_consulta(self, consulta: str) -> pd.DataFrame:

        prompt = self._prompt_base.format(consulta)
        consulta_sparql = self._lm.generar_respuesta(prompt).strip()

        try:
            resultados = self._graph.query(consulta_sparql)
        except Exception as e:
            print(f"Error ejecutando SPARQL: {e}")
            return pd.DataFrame()

        # Extraer los nodos que aparecen en los resultados
        valores = set()
        for row in resultados:
            for val in row:
                if isinstance(val, URIRef):
                    nombre = str(val).split("/")[-1].replace("_", " ")
                    valores.add(nombre)

        # Filtrar el DataFrame original por coincidencias en Sujeto u Objeto
        df_filtrado = self._data[
            self._data["Sujeto"].isin(valores) | self._data["Objeto"].isin(valores)
        ]
        return df_filtrado.reset_index(drop=True)

In [ ]:
bd_grafo = BDGraph(df_relaciones, llm_mistral)

# Clasificador de consultas

Se compararán dos clasificadores y se evaluará su performance:
- Evaluador con Regresión Logística
- Evaluador LLM con entrenamiento few-shot

In [ ]:
# Procesamiento de datos
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Datos de entrenamiento y prueba

### Archivo CSV de consultas

Generado con Claude Sonnet 4

In [ ]:
df_queries = pd.read_csv("datos/queries_clasificador.csv")
X_train, X_test, y_train, y_test = train_test_split(df_queries["query"], df_queries["categoria"], test_size=0.2, random_state=42)

## Clasificador LR

Se hará embedding del texto utilizando el modelo `e5-multilingual-small`.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
class Clasificador_LR():
    def __init__(self, X_train_string, y_train, tokenizer, embedder, random_state=None):
        self._model = LogisticRegression(class_weight='balanced', random_state=random_state)

        self._tokenizer = tokenizer
        self._embedder = embedder
        X_train_embed = self._embed_sentences(X_train_string)

        self._model.fit(X_train_embed, y_train)

    def _embed_sentences(self, sentences):
        inputs = self._tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            model_output = self._embedder(**inputs)
        embeddings = model_output.last_hidden_state.mean(dim=1)  # Mean pooling
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)  # L2 normalize
        return embeddings.cpu().numpy()

    def predecir(self, data):
        # Si es un único valor, convertir en lista
        if isinstance(data, str):
            print("is string")
            data = [data]
        text_embedded = self._embed_sentences(data)
        return self._model.predict(text_embedded)

In [ ]:
clasificador_lr = Clasificador_LR(X_train.to_list(), y_train, tokenizer, embedder, random_state=42)


In [ ]:
print(classification_report(y_test, clasificador_lr.predecir(X_test.to_list())))
clasificador_lr.predecir("¿Quién inventó el juego?")[0]

              precision    recall  f1-score   support

estadisticas       0.64      0.62      0.63        26
 informacion       0.92      0.90      0.91       101
  relaciones       0.65      0.79      0.71        14

    accuracy                           0.84       141
   macro avg       0.74      0.77      0.75       141
weighted avg       0.84      0.84      0.84       141

is string


'relaciones'

## Clasificador LLM

In [ ]:
class Clasificador_LLM():
    def __init__(self, lm: LanguageModel):
        self._lm = lm
        self._prompt_base = self._crear_prompt_base()

    def _crear_prompt_base(self):
        return """
Eres un modelo de clasificacion de consultas sobre un juego llamado Tiny Towns. Debes clasificar las consultas en una de las siguientes 3 categorías:
1. relaciones -> Preguntas sobre algo que sea una relación sobre temas alrededor del juego, su entorno, como sus creadores, otros juegos, etc.
2. informacion -> Preguntas sobre información del juego, como reglas, mecánicas, qué hacer en ciertas situaciones
3. estadisticas -> Preguntas sobre datos numéricos o estadísticos del juego, como números, precios, cantidad de jugadores o de ventas, dificultad asociada etc.

Por ejemplo:
Pregunta: 'Quien es el creador del juego?', Respuesta: relaciones
Pregunta: 'Qué juego relacionado existe?', Respuesta: relaciones
Pregunta: 'Como se gana?', Respuesta: informacion
Pregunta: 'Qué hacer si pierdo mi ultima ficha?', Respuesta: informacion
Pregunta: 'Que es lo más divertido de Tiny Towns?', Respuesta: informacion
Pregunta: 'Cuantos jugadores admite el juego?', Respuesta: estadisticas
Pregunta: 'Cuantos jugadores activos tiene el juego?', Respuesta: estadisticas
Pregunta: 'Cuál es el precio en Amazon?', Respuesta: estadisticas

Tu consulta es la siguiente:
{}

Responde solo con una de las siguientes palabras exactas: 'relaciones', 'informacion' o 'estadisticas'. NO RESPONDER NADA MAS QUE LA CATEGORÍA.
"""

    def predecir(self, data):
        if isinstance(data, str):
            data = [data]

        predicciones = []
        for query in data:
            prompt = self._prompt_base.format(query)
            respuesta = self._lm.generar_respuesta(prompt).replace('```python', '').replace('```', '').strip().lower()
            # Asegura que la respuesta sea válida
            if respuesta not in {"relaciones", "informacion", "estadisticas"}:
                print(f"Respuesta inválida del clasificador LLM: {respuesta}")
                respuesta = "informacion"  # por defecto si no es válida
            predicciones.append(respuesta)

        return predicciones

In [ ]:
clasificador_llm = Clasificador_LLM(llm_mistral)

print(classification_report(y_test[:5], clasificador_llm.predecir(X_test[:5].to_list())))
clasificador_llm.predecir("¿Quién inventó el juego?")[0]

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

estadisticas       0.50      1.00      0.67         1
 informacion       1.00      0.50      0.67         4
  relaciones       0.00      0.00      0.00         0

    accuracy                           0.60         5
   macro avg       0.50      0.50      0.44         5
weighted avg       0.90      0.60      0.67         5



'relaciones'

## Conclusión y elección del modelo

Se puede observar que el clasificador por regresión logística tiene un performance aceptable, y un costo mucho menor. Por lo tanto, se utilizará este clasificador para el modelo final.

Sin embargo, es razonable suponer que el clasificador LLM tendrá mayores capacidades de generalización al estar pre-entrenado y ser de uso general, pudiendo razonar de forma zero-shot la categoría a asignar a consultas diferentes a cualquier otra en el dataset. Ahora bien, incluso a pesar de esto, no resulta una opción fácil de probar, y posiblemente no viable al consumir demasiado rápidamente la cuota de prompts que nos proporcionan los servicios de LLMs por APIs.

# Pipeline de Recuperación

In [ ]:
!pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
class BM25Searcher():
    def __init__(self, texto):
        self.texto = texto
        self.texto_token = [word_tokenize(t.lower()) for t in texto]
        self.bm25 = BM25Okapi(self.texto_token)

    def buscar(self, query, n_resultados=5):
        tokenized_query = word_tokenize(query.lower())
        scores = self.bm25.get_scores(tokenized_query)
        ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:n_resultados]
        return [self.texto[i] for i in ranked_indices]

class BDVectorial_Hibrida():
    def __init__(self, bd_1: BDVectorial, bd_2: BM25Searcher):
        self._vector_searcher = bd_1
        self._bm25_searcher = bd_2

    def ejecutar_consulta(self, consulta, n_resultados=5):
        sem_docs = self._vector_searcher.buscar(consulta, n_resultados)
        bm_docs = self._bm25_searcher.buscar(consulta, n_resultados)

        # Combinar resultados por documento
        combined_docs = list(set(sem_docs + bm_docs))

        # rerank?

        return combined_docs

bm25_searcher = BM25Searcher(textos_split)
bd_vectorial_hibrida = BDVectorial_Hibrida(bd_vectorial, bm25_searcher)

In [ ]:
class Recuperador():
    """
    Recuperador de información a partir de las bases de datos para RAG.
    """
    def __init__(self, bd_info: BDVectorial_Hibrida, bd_stats: BDTabular, bd_rels: BDGraph, clasificador):
        self._bd_info = bd_info
        self._bd_stats = bd_stats
        self._bd_rels = bd_rels
        self._clasificador = clasificador

    def recuperar(self, query: str):
        clase = self._clasificador.predecir(query)[0]
        if clase == "informacion":
            return self._bd_info.ejecutar_consulta(query, n_resultados=5)
        elif clase == "estadisticas":
            return self._bd_stats.ejecutar_consulta(query)
        elif clase == "relaciones":
            return self._bd_rels.ejecutar_consulta(query)

recuperador = Recuperador(bd_info=bd_vectorial_hibrida, bd_stats=bd_tabular, bd_rels=bd_grafo, clasificador=clasificador_lr)

# Chatbot

In [ ]:
class Chatbot():
    def __init__(self, recuperador: Recuperador, llm: LanguageModel):
        self._recuperador = recuperador
        self._llm = llm

    def chat_loop(self):
        prompt = self._generar_prompt_inicial()
        print("¡Hola! Soy el chatbot de Tiny Towns. ¿En qué puedo ayudarte hoy?")
        while True:
            query = input("> ")
            if query.lower() in ["salir", "exit", "quit"]:
                print("¡Hasta luego!")
                break

            contexto_rag = self._recuperador.recuperar(query)

            prompt = prompt.format(query, contexto_rag)

            respuesta = self._llm.generar_respuesta(prompt)

            print("-- COMIENZA RESPUESTA")
            print(respuesta)
            print("-- FIN RESPUESTA")

            prompt = self._continuar_prompt_contexto(query, respuesta)



    def _generar_prompt_inicial(self):
        return """
Eres un asistente útil para responder preguntas sobre el juego de mesaTiny Towns.
Para cada pregunta se te proporcionará información relevante para responderla de una Base de Datos.

<INICIA CONVERSACION>

Tú: ¡Hola! Soy el chatbot de Tiny Towns. ¿En qué puedo ayudarte hoy?

CONSULTA: {}

INFORMACION RELEVANTE (IMPORTANTE: Si la información y el contexto no fueran suficientes para responder a la pregunta, debes pedir al usuario que reformule su pregunta.): {}

Responde la consulta del usuario.
"""

    def _continuar_prompt_contexto(self, query_anterior, respuesta):
        return query_anterior + "\n\n Tú:" + respuesta + """

<SIGUIENTE CONSULTA>

CONSULTA: {}

INFORMACION RELEVANTE: {}

IMPORTANTE: Si la información y el contexto no fueran suficientes para responder a la pregunta, debes pedir al usuario que reformule su pregunta.

Responde la consulta del usuario.
"""

In [ ]:
Chatbot(recuperador, llm_mistral).chat_loop()

¡Hola! Soy el chatbot de Tiny Towns. ¿En qué puedo ayudarte hoy?
> Como gano una partidita


KeyError: 'self'